In [1]:
%matplotlib inline

# To reload external scripts automatically
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

# Importing external files
import sys
sys.path.append('scripts/')
from data_import import *
from similarities import *

# 0-Exploratory
This notebook contain the first study of the data to identify what is needed and what we do not need. It will only work with subsets of datasets as they will not fit into memory.

In [2]:
DATA_FOLDER = "../../Project-Data/"
META_FOLDER = DATA_FOLDER + "meta/"
REVIEWS_FOLDER = DATA_FOLDER + "reviews/"
DUMP_FOLDER = DATA_FOLDER + "dump/"
CATEGORIES = ['Books', 'CDs_and_Vinyl', 'Electronics', 'Movies_and_Tv']
MAXCOUNT = -1

## 0.0-Content of the files

The different columns of the **metadata** files are : 
* ```asin``` : the unique identifier of the object
* ```brand```
* ```categories``` : the categories of the object
* ```description``` : the description of the object
* ```imUrl```  : the link toward the images related to the object
* ```price```
* ```related``` : a list of objects that are related to this object
* ```salesRank``` 
* ```title```

The different columns of the **reviews** files are :
* ```asin``` : the unique identifier of the object
* ```helpful``` : a list of 2 integers [x,y], the helpfulness score is x/y votes
* ```overall``` : the rating of the object
* ```reviewText```
* ```reviewTime```
* ```reviewerID```
* ```reviewerName```
* ```summary ``` : the title of the review
* ```unixReviewTime``` : in Unix format

Therefore we keep only the column that are of interest for our task

In [3]:
meta_interesting_cols = ['asin', 'title', 'salesRank', 'description']
review_interesting_cols = ['asin', 'overall', 'unixReviewTime']

## 0.1-Books

In [4]:
meta_books_path, review_books_path = get_paths(0, DATA_FOLDER, META_FOLDER,
                                               REVIEWS_FOLDER, CATEGORIES)

### 0.1.1-Metadata

In [6]:
meta_books = import_interesting_cols(meta_books_path,DUMP_FOLDER,True,meta_interesting_cols,max_count=MAXCOUNT)
meta_books.head()

Retrieving from : ../../Project-Data/dump/meta_Books_asin_title_salesRank_description_ALL
It took 00:00:03.731 to import the data.


,asin,description,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Books,salesRank_Cell_Phones_&_Accessories,salesRank_Clothing,salesRank_Electronics,salesRank_Health_&_Personal_Care,salesRank_Home_&_Kitchen,salesRank_Industrial_&_Scientific,salesRank_Jewelry,salesRank_Kitchen_&_Dining,salesRank_Movies_&_TV,salesRank_Music,salesRank_Musical_Instruments,salesRank_Office_Products,salesRank_Shoes,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,title
0,0001048791,NaN,NaN,6334800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Crucible: Performed by Stuart Pankin, Jero..."
1,0001048775,William Shakespeare is widely regarded as the ...,NaN,13243226.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Measure for Measure: Complete & Unabridged
2,0001048236,"""One thing is certain, Sherlockians, put aside...",NaN,8973864.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Sherlock Holmes Audio Collection
3,0000401048,NaN,NaN,6448843.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The rogue of publishers' row;: Confessions of ...
4,0001019880,NaN,NaN,9589258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Classic Soul Winner's New Testament Bible


### 0.1.2-Reviews

In [37]:
review_books = import_interesting_cols(
    review_books_path, False, review_interesting_cols, max_count=MAXCOUNT)
review_books.head()

,asin,overall,unixReviewTime
0,0000000116,4.0,2002-04-27
1,0000000116,1.0,2014-03-24
2,0000000868,4.0,2002-09-11
3,0000013714,4.0,2013-10-31
4,0000013714,5.0,2013-07-27


## 0.2-CDs and Vinyls

In [56]:
meta_cd_path, review_cd_path = get_paths(1, DATA_FOLDER, META_FOLDER,
                                         REVIEWS_FOLDER, CATEGORIES)

### 0.2.1-Metadata

In [39]:
meta_cds = import_interesting_cols(
    meta_cd_path, True, meta_interesting_cols, max_count=MAXCOUNT, dropna=False)
meta_cds.head()

,asin,description,salesRank_Movies_&_TV,salesRank_Music,title
0,0001501348,"Lenny LeBlanc, Alex Acuna, Justo Almario, Tom ...",359265.0,NaN,Lift Him Up With Ron Kenoly [VHS]
1,0001393774,Audio CD,NaN,41017.0,Songs for the Shepherd
2,0005123909,18 Music Videos for Kids: Do Your Ears Hang Lo...,451209.0,NaN,Silly Songs: 18 Wholesome Fun Songs for Kids [...
3,0005072298,NaN,NaN,350804.0,Hymns: 16 Classic Hymns for Children
4,0005224896,NaN,NaN,347825.0,"Voice of the Wind: Personal Worship, Vol. 1"


### 0.2.2-Reviews

In [40]:
review_cds = import_interesting_cols(
    review_cd_path,
    False,
    review_interesting_cols,
    max_count=100,
    dropna=False)
review_cds.head()

,asin,overall,unixReviewTime
0,0001393774,5.0,2013-08-31
1,0001393774,5.0,2013-07-02
2,0001393774,5.0,2014-04-02
3,0001393774,5.0,2014-02-15
4,0001393774,5.0,2005-11-01


## 0.3-Electronics

In [57]:
meta_electronic_path, review_electronic_path = get_paths(
    2, DATA_FOLDER, META_FOLDER, REVIEWS_FOLDER, CATEGORIES)

### 0.3.1-Metadata

In [58]:
meta_electronic = import_interesting_cols(
    meta_electronic_path,
    True,
    meta_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
meta_electronic.head(2)

,asin,description,salesRank_Cell_Phones_&_Accessories,salesRank_Electronics,salesRank_Software,salesRank_Sports_Outdoors,title
0,0132793040,The Kelby Training DVD Mastering Blend Modes i...,NaN,NaN,NaN,NaN,Kelby Training DVD: Mastering Blend Modes in A...
1,0321732944,NaN,NaN,NaN,NaN,NaN,Kelby Training DVD: Adobe Photoshop CS5 Crash ...


### 0.3.2-Review

In [48]:
review_electronic = import_interesting_cols(
    review_electronic_path,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
review_electronic.head()

,asin,overall,unixReviewTime
0,0132793040,5.0,2013-04-13
1,0321732944,5.0,2012-07-01
2,0439886341,1.0,2013-04-29
3,0439886341,3.0,2013-07-22
4,0439886341,1.0,2012-04-18


## 1-Detecting similar products
In this section we try to obtain the products that are similar among a given category and to develop a systematic way to detect those similar products.

## 1.1-Books

Some books are sold in different formats : hard cover, pocket, electronic, etc. Therefore in order to compare them we will focus on the title and description that are the only two attributes that should be highly similar between the two different yet similar books.

In [8]:
meta_books = import_interesting_cols(meta_books_path,True,meta_interesting_cols,max_count=MAXCOUNT)
meta_books.head()

,asin,description,salesRank_Books,title
0,0001048791,NaN,6334800.0,"The Crucible: Performed by Stuart Pankin, Jero..."
1,0001048775,William Shakespeare is widely regarded as the ...,13243226.0,Measure for Measure: Complete & Unabridged
2,0001048236,"""One thing is certain, Sherlockians, put aside...",8973864.0,The Sherlock Holmes Audio Collection
3,0000401048,NaN,6448843.0,The rogue of publishers' row;: Confessions of ...
4,0001019880,NaN,9589258.0,Classic Soul Winner's New Testament Bible


In [17]:
list(meta_books['title'])[:100]

['The Crucible: Performed by Stuart Pankin, Jerome Dempsey & Cast',
 'Measure for Measure: Complete & Unabridged',
 'The Sherlock Holmes Audio Collection',
 "The rogue of publishers' row;: Confessions of a publisher (A Banner Book)",
 "Classic Soul Winner's New Testament Bible",
 'Archer Christmas 4 Tape Pack',
 'Sonatas - For Piano',
 'Classic Connolly Boxed Set (Vol 1 & 2)',
 'Hand in Glove',
 'War Poems: An Anthology of Poetry from the 18th Cantury to the Present Day',
 'The Way Things Work: An Illustrated Encyclopedia of Technology',
 'As You Like it: Complete & Unabridged',
 'Pearl and Sir Orfeo',
 'Master Georgie',
 'Celebremos Su Gloria',
 'Hamlet: Complete & Unabridged',
 'The Laurel & Hardy Book (The Curtis Film Series)',
 'Classic Connolly in Words and Music (HarperCollinsComedy)',
 'One, Two, Guess Who?',
 'Foot Rot of Piper nigrum. L. (Phytopathological Paper)',
 'Who on Earth is Tom Baker?',
 "Mog's Kittens",
 'Double-Speak: From Revenue Enhancement to Terminal Living--How

In [7]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    toReturn = stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))
    return toReturn

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

[nltk_data] Downloading package punkt to /Users/hmoreau/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
print(cosine_sim('a little bird', 'a little bird chirps'))

[[ 1.          0.70929727]
 [ 0.70929727  1.        ]]


In [ ]:
print(cosine_sim('a little bird', 'a little bird'))
print(cosine_sim('a little bird', 'a little bird chirps'))
print(cosine_sim('a little bird', 'a big dog barks'))